In [1]:
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from create_model import create_model
from create_dataset import TrainDataset

2022-06-30 19:17:09.345810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 19:17:09.357145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 19:17:09.357570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 19:17:09.359191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [2]:
BATCH_SIZE=128
KERNEL_SIZE=4
ACTIVATION_FUNCTION='relu'
LEARNING_RATE=0.001
NUM_UNITS_DENSE=256
NUM_UNITS_LTSM2=1024
### run 55 parameters
DROPOUT=0.12489316869910207
NUM_UNITS_LTSM1=512
### run 41 parameters
# DROPOUT=0.1
# NUM_UNITS_LTSM1=768

In [3]:
model_path = 'gs://iam-model-staging/run_55_train/model/'
pred_model = keras.models.load_model(model_path)

pred_model.save_weights(Path('checkpoints', 'pred'))

In [6]:
full_model = create_model(KERNEL_SIZE, ACTIVATION_FUNCTION, NUM_UNITS_DENSE, DROPOUT, NUM_UNITS_LTSM1, NUM_UNITS_LTSM2, LEARNING_RATE)
full_model.load_weights(Path('checkpoints', 'pred'))
for i in range(len(full_model.layers)):
    full_model.layers[i].trainable=False
full_model.layers[6].trainable = True
full_model.layers[-1].trainable = True
full_model.compile(keras.optimizers.Adam(learning_rate=LEARNING_RATE))

2022-06-30 19:22:42.940146: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.00MiB (rounded to 16777216)requested by op RandomUniform
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-30 19:22:42.940219: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-06-30 19:22:42.940235: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 28, Chunks in use: 28. 7.0KiB allocated for chunks. 7.0KiB in use in bin. 1.6KiB client-requested in use in bin.
2022-06-30 19:22:42.940246: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.2KiB allocated for chunks. 1.2KiB in use in bin. 980B client-requested in use in bin.
2022-06-30 19:22:42.940257: I tens

ResourceExhaustedError: OOM when allocating tensor with shape[1024,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
# load Steyermark or Standley images
folder = Path('images/standley_4058_train')
metadata = Path('words_metadata.csv')
data = TrainDataset()
data.create_dataset(128, folder, metadata)

# fit open layers
history = full_model.fit(data.train_dataset, epochs=50, validation_data=data.validation_dataset)

full_model.save('retrained.model')

prediction_model = tf.keras.models.Model(
    full_model.get_layer(name='image').input, full_model.get_layer(name='dense_layer').output
)
prediction_model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
prediction_model.save('retrained-prediction.model')

In [ ]:
# fine tune
full_model.trainable = True
full_model.compile(keras.optimizers.Adam(learning_rate=1e-5))
history_fine_tune = full_model.fit(data.train_dataset, epochs=25, validation_data=data.validation_dataset)


full_model.save('fine_tuned.model')

prediction_model = tf.keras.models.Model(
    full_model.get_layer(name='image').input, full_model.get_layer(name='dense_layer').output
)
prediction_model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
prediction_model.save('fine_tuned-prediction.model')